<a id = '4.0'></a>
<p style = "font-size : 35px; color : #34656d ;  text-align : center; background-color : #08f9f9; border-radius: 5px 5px;"><strong>Regularization Part_3 (Elastic Net- Gridsearch)</strong></p> 

In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10,6)
pd.set_option('display.max_columns', 100)

In [89]:
df = pd.read_csv("Advertising.csv")

In [90]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [91]:
df.shape

(200, 4)

In [92]:
X = df.drop("sales", axis =1)
y = df["sales"]

In [93]:
X.head()

,TV,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4


### Polynomial Features

In [94]:
from sklearn.preprocessing import PolynomialFeatures

We create an artificial overfiting situation by taking poly feature from 5 degrees

In [95]:
polynomial_converter = PolynomialFeatures(degree=5, include_bias=False)

poly_features = polynomial_converter.fit_transform(X)

# Burada suni olarak bir overfitting durumu oluşturuyoruz . Sonra bunu çözeceğiz.

In [96]:
poly_features.shape

(200, 55)

### Train | Test Split

In [97]:
from sklearn.model_selection import train_test_split

In [98]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

## Scaling the Data

In [103]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [104]:
scaler.fit(X_train)  # Apply just for X_tarin not for X_test (Kalıp çıkarma)

StandardScaler()

In [105]:
X_train_scaled = scaler.transform(X_train) # Apply transform according to fit
X_test_scaled = scaler.transform(X_test)

## <span style='color:red'> Elastic net</span> 

In [18]:
from sklearn.linear_model import ElasticNet, ElasticNetCV

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_val(y_train, y_train_pred, y_test, y_pred, name):
    
    scores = {name+"_train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    name+"_test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [106]:
elastic_model = ElasticNet(alpha=1, l1_ratio=0.5, random_state=42) # l1_ratio: 1: Lasso or 0:Ridge
elastic_model.fit(X_train_scaled, y_train)

ElasticNet(alpha=1, random_state=42)

In [107]:
y_pred = elastic_model.predict(X_test_scaled)
y_train_pred = elastic_model.predict(X_train_scaled)

In [108]:
es = train_val(y_train, y_train_pred, y_test, y_pred, "elastic")
es
# Default değerler kullandığımız için sonuçlar çok iyi değil.

,elastic_train,elastic_test
R2,0.896126,0.902016
mae,1.248480,1.200654
mse,2.733565,2.766287
rmse,1.653350,1.663216


In [109]:
# pd.concat([ls,rs, rcs, lss, lcs, es], axis = 1)

#  linear_train	linear_test	ridge_train	ridge_test	ridge_cv_train	ridge_cv_test	lasso_train	lasso_test	lasso_cv_train	lasso_cv_test	elastic_train	elastic_test
# R2	0.997607	0.764990	0.988677	0.982511	0.994491	0.983643	0.919650	0.918590	0.988824	0.986295	0.896126	0.902016
# mae	0.186213	0.665959	0.338149	0.482446	0.244842	0.442087	1.018355	1.017959	0.339065	0.440975	1.248480	1.200654
# mse	0.062968	6.634847	0.297970	0.493743	0.144977	0.461803	2.114491	2.298390	0.294098	0.386919	2.733565	2.766287
# rmse	0.250934	2.575820	0.545866	0.702669	0.380758	0.679561	1.454129	1.516044	0.542308	0.622028	1.653350	1.663216


## ElasticNet Cross Validation (Default parameters: alpha = 1 and l1_ratio=0.5)

In [ ]:
from sklearn.model_selection import cross_validate

In [23]:
model = ElasticNet(alpha=1, l1_ratio=0.5, random_state=42)
scores = cross_validate(model, X_train_scaled, y_train,
                        scoring=['r2', 'neg_mean_absolute_error','neg_mean_squared_error','neg_root_mean_squared_error'], cv=5)

In [24]:
scores = pd.DataFrame(scores, index = range(1, 6))
scores.iloc[:,2:]

,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.915497,-1.255282,-2.567362,-1.602299
2,0.928138,-0.960011,-1.620225,-1.272881
3,0.855390,-1.373843,-4.014250,-2.003559
4,0.875648,-1.431847,-3.384817,-1.839787
5,0.840262,-1.449269,-2.953597,-1.718603


In [25]:
scores = pd.DataFrame(scores, index = range(1, 11))
scores.iloc[:,2:].mean()
# Cross-Validation sonrası score umuzu 0.88

test_r2                             0.882987
test_neg_mean_absolute_error       -1.294050
test_neg_mean_squared_error        -2.908050
test_neg_root_mean_squared_error   -1.687426
dtype: float64

In [26]:
train_val(y_train, y_train_pred, y_test, y_pred, "elastic")

,elastic_train,elastic_test
R2,0.896126,0.902016
mae,1.248480,1.200654
mse,2.733565,2.766287
rmse,1.653350,1.663216


In [27]:
elastic_model.coef_

array([ 0.8629361 ,  0.2114429 ,  0.        ,  0.29076735,  0.98099122,
        0.00249769,  0.        ,  0.        ,  0.        ,  0.        ,
        0.52021768,  0.        ,  0.6023984 ,  0.09615974,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.15970242,  0.        ,  0.27423182,  0.        ,  0.        ,
        0.3407246 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.0826519 ,  0.        ,  0.        ,  0.        ,  0.15709256,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ])

In [28]:
em_df = pd.DataFrame(elastic_model.coef_, columns=["elastic_coef_(alp:1, l1:0.5)"])
em_df

,"elastic_coef_(alp:1, l1:0.5)"
0,0.862936
1,0.211443
2,0.000000
3,0.290767
4,0.980991
5,0.002498
6,0.000000
7,0.000000
8,0.000000
9,0.000000


### Choosing best alpha and l1_ratio values by ElasticNetCV

In [110]:
alpha_space = np.linspace(0.01, 1, 100)
alpha_space

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [122]:
elastic_cv_model = ElasticNetCV(alphas = alpha_space, l1_ratio=[0.1, 0.5, 0.7,0.9, 0.95, 1], cv = 5, 
                                max_iter = 100000,random_state=42)
# burada l1_ratio hyperparameter' i yeni. Burada küçük bir array verdik. Çok değer vermeye gerek yok. (0-1 arası random değer.)
# Bunlardan uygun olanı bulacak.

In [123]:
elastic_cv_model.fit(X_train_scaled, y_train)

ElasticNetCV(alphas=array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ]),
             cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 1], max_iter=100000,
             random_state=42)

In [124]:
elastic_cv_model.alpha_
# En uygun alpha değeri 0.01 imiş.
# alpha  = yapılacak regülarizasyonun kuvveti

0.01

In [125]:
elastic_cv_model.l1_ratio_
# En uygun l1_ratio = 1.0 çıktı. Yani bize Lasso kullan dedi.
# l1 = lasso, l2 = ridge.

1.0

In [126]:
y_pred = elastic_cv_model.predict(X_test_scaled)
y_train_pred = elastic_cv_model.predict(X_train_scaled)

In [127]:
ecs = train_val(y_train, y_train_pred, y_test, y_pred, "elastic_cv")
ecs

,elastic_cv_train,elastic_cv_test
R2,0.988824,0.986295
mae,0.339065,0.440975
mse,0.294098,0.386919
rmse,0.542308,0.622028


In [128]:
pd.concat([es, ecs], axis = 1)
# parametreleri en optimim parametrelere getirince score arttı. (elastic_cv_test score =0.98)

,elastic_train,elastic_test,elastic_cv_train,elastic_cv_test
R2,0.896126,0.902016,0.988824,0.986295
mae,1.248480,1.200654,0.339065,0.440975
mse,2.733565,2.766287,0.294098,0.386919
rmse,1.653350,1.663216,0.542308,0.622028


In [71]:
elastic_cv_model.coef_

array([ 4.27021377,  0.18037374,  0.13965856, -3.14605513,  4.15339959,
       -0.        ,  0.        ,  0.06249654,  0.        , -0.        ,
       -0.43049816, -0.17433788,  0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.4873868 ,
       -0.        ,  0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.06929818,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ])

In [72]:
ecm_df = pd.DataFrame(elastic_cv_model.coef_, columns=["elastic_coef_(alp:0.01, l1:1)"])

In [73]:
pd.concat([em_df, ecm_df], axis = 1)

,"elastic_coef_(alp:1, l1:0.5)","elastic_coef_(alp:0.01, l1:1)"
0,0.862936,4.270214
1,0.211443,0.180374
2,0.000000,0.139659
3,0.290767,-3.146055
4,0.980991,4.153400
5,0.002498,-0.000000
6,0.000000,0.000000
7,0.000000,0.062497
8,0.000000,0.000000
9,0.000000,-0.000000


## <span style='color:red'> Grid Search</span> 

In [74]:
from sklearn.model_selection import GridSearchCV
# RidgeCV, LassoCV, ElasticCV yerine Grid Search uygulayacağız. Bu tüm modellerde geçerlidir.
# Bu işlem data boyutuna bağlı olarak çok uzun zaman alabilir.

In [75]:
elastic_model = ElasticNet(max_iter=10000, random_state=42)

In [76]:
param_grid = {"alpha":[0.01, 0.012, 0.2, 0.5, 0.6, 0.7, 1],
            "l1_ratio":[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]}
# burada alpha ve l1_ratio için çok daha fazla değer verebilirsiniz. 
# np.linspace ile çok daha fazla değer verilebilir.

In [77]:
grid_model = GridSearchCV(estimator = elastic_model, param_grid = param_grid, scoring = 'neg_root_mean_squared_error',
                         cv =5, verbose =2)
# verbose =2 ( her 2 iterasyonda 1 bana sonuçları göster anlamına gelmektedir.)
# estimator = elastic_model(Elastic Net seçiyoruz ki lasso ve ridge ikisi de değerlendirilsin.)

In [78]:
grid_model.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 49 candidates, totalling 245 fits
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.1s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.5; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.7; total time=   0.0s
[CV] END ...........................alpha=0.01,

GridSearchCV(cv=5, estimator=ElasticNet(max_iter=10000, random_state=42),
             param_grid={'alpha': [0.01, 0.012, 0.2, 0.5, 0.6, 0.7, 1],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [79]:
grid_model.best_params_
# en uygun alpha 0.01 imiş ve l1_ratio 1 imiş. Yani tamamında Lasso kullanılsın diyor.
# l1_ratio = 0.3 olsa idi %30 lasso %70 ridge kullanılacaktı.

{'alpha': 0.01, 'l1_ratio': 1}

In [80]:
pd.DataFrame(grid_model.cv_results_)
# Burada bütün sonuçları görebiliriz.
# Burada en sağdaki sütunda rank_test_score'a bakarsak burada 1 olan en iyi sonuçtur.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.036212,0.021294,0.000741,0.000223,0.01,0.1,"{'alpha': 0.01, 'l1_ratio': 0.1}",-0.586201,-0.693806,-1.184123,-0.491352,-0.490280,-0.689152,0.258570,13
1,0.014858,0.011403,0.000881,0.000284,0.01,0.5,"{'alpha': 0.01, 'l1_ratio': 0.5}",-0.553277,-0.537915,-1.172816,-0.488299,-0.493055,-0.649072,0.263070,10
2,0.010603,0.007102,0.000728,0.000187,0.01,0.7,"{'alpha': 0.01, 'l1_ratio': 0.7}",-0.529432,-0.527849,-1.157311,-0.480464,-0.500409,-0.639093,0.259749,9
3,0.008731,0.001245,0.000847,0.000330,0.01,0.9,"{'alpha': 0.01, 'l1_ratio': 0.9}",-0.490240,-0.504487,-1.120156,-0.446524,-0.484072,-0.609096,0.256247,6
4,0.020665,0.023390,0.001006,0.000363,0.01,0.95,"{'alpha': 0.01, 'l1_ratio': 0.95}",-0.470398,-0.497256,-1.110933,-0.437316,-0.481998,-0.599580,0.256435,3
5,0.016159,0.008445,0.003733,0.003054,0.01,0.99,"{'alpha': 0.01, 'l1_ratio': 0.99}",-0.455118,-0.493277,-1.098376,-0.430248,-0.484677,-0.592339,0.254000,2
6,0.009399,0.002504,0.002299,0.001205,0.01,1,"{'alpha': 0.01, 'l1_ratio': 1}",-0.451422,-0.492426,-1.094927,-0.428596,-0.486989,-0.590872,0.253114,1
7,0.013456,0.003921,0.000680,0.000124,0.012,0.1,"{'alpha': 0.012, 'l1_ratio': 0.1}",-0.606883,-0.654207,-1.214541,-0.509165,-0.495180,-0.695995,0.266011,14
8,0.008655,0.003246,0.001153,0.000605,0.012,0.5,"{'alpha': 0.012, 'l1_ratio': 0.5}",-0.573940,-0.553763,-1.195471,-0.509639,-0.498213,-0.666205,0.266088,12
9,0.007627,0.005334,0.000658,0.000128,0.012,0.7,"{'alpha': 0.012, 'l1_ratio': 0.7}",-0.552026,-0.551934,-1.180732,-0.504933,-0.506986,-0.659322,0.261517,11


In [81]:
grid_model.best_index_
# index numarası 6 olan durumda en iyi score alınıyormuş. 

6

In [82]:
grid_model.best_score_
# en iyi score 0.59 imiş

-0.5908719878682445

## Using Best Hyperparameters from GridSearch

In [83]:
y_pred = grid_model.predict(X_test_scaled)
y_train_pred = grid_model.predict(X_train_scaled)

In [84]:
train_val(y_train, y_train_pred, y_test, y_pred, "GridSearch")
# Bütün işlemler sonucunda elde edilen en iyi sonuçlar bunlar.

,GridSearch_train,GridSearch_test
R2,0.988824,0.986295
mae,0.339065,0.440975
mse,0.294098,0.386919
rmse,0.542308,0.622028


## <span style='color:Blue'> Final Model</span> 

In [85]:
final_scaler = StandardScaler()
X_scaled = final_scaler.fit_transform(poly_features)
# Final model'de artık datanın hepsini kullanıyoruz(poly_features)

In [86]:
from sklearn.linear_model import Lasso
final_model = Lasso(alpha=0.01) #lasso_cv_model
# alpha=0.01 ve lasso için en iyi score alacağımızı GridSearch ile öğrenmiştik.

In [87]:
final_model.fit(X_scaled, y)
# Final modeli tüm data üzerinden eğittik.

Lasso(alpha=0.01)

In [ ]:
#Bu modelin son halini saklıyoruz. Belki bir websitesi üzerinden hizmet verecek...
# Kullanıcı burada radyo, tv, gazete için reklam harcaması girecek ve satıiş miktarındaki değişimleri görecek.
# Bu verilere göre karar alacak.
# Bütün modellerde bu şekilde yapacağız.
# Bu modelleri deploy etmeyi Machine Learning - Deployment kısmında göreceğiz.
# Sürekli yeni data girişi varsa o dataları ekleyip modeli refresh ederek yeniden deploy etmek gerekir.